**Inteligência Artificial - T1** \\
Prof. Eraldo Luís Rezende Fernandes \\
FACOM – UFMS – 2020.1 \\
Aluno: Pedro Luiz da Costa Silva - RGA 2018.1904.032-5 \\

**Trabalho 2**

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import loguniform, uniform

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import preprocessing

rnd = np.random.RandomState(42)

In [6]:
df_train = pd.read_csv("nave_train.csv")
df_test = pd.read_csv("nave_test.csv")

print(df_train.shape)
print(df_test.shape)

df_train.sample(5)

(712, 12)
(179, 11)


,IdPassageiro,Sobreviveu,ClassePassageiro,Nome,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Tarifa,Cabine,Embarque
210,3,sim,3,"Heikkinen, Srta. Laina",feminino,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
297,467,nao,2,"Campbell, Sr. William",masculino,NaN,0,0,239853,0.0000,NaN,S
383,193,sim,3,"Andersen-Jensen, Srta. Carla Christine Nielsine",feminino,19.0,1,0,350046,7.8542,NaN,S
264,443,nao,3,"Petterson, Sr. Johan Emil",masculino,25.0,1,0,347076,7.7750,NaN,S
672,659,nao,2,"Eitemiller, Sr. George Floyd",masculino,23.0,0,0,29751,13.0000,NaN,S


# Tratamento dos dados



## Remoção de atributos irrelevantes
Removidos:
* Nome: Não influencia;
* IdPassageiro: Considerando que números de ID são gerados aleatoriamente, não influencia;
* Tarifa: Talvez pudesse influenciar (pessoas que pagaram mais caro podem ter lugares com maiores chances de sobrevivência), mas nos testes ela abaixava a acurácia;
* Cabine: Talvez pudesse influenciar, mas nos testes abaixava a acurácia.

In [7]:
colunasdrop = ['Nome', 'IdPassageiro', 'Tarifa', 'Cabine']

df_train = df_train.drop(columns = colunasdrop);
df_test = df_test.drop(columns = colunasdrop);

df_train.sample(5)

,Sobreviveu,ClassePassageiro,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Embarque
523,nao,3,masculino,NaN,0,0,2641,C
24,sim,3,feminino,0.75,2,1,2666,C
494,nao,2,masculino,36.00,0,0,SC/Paris 2163,C
447,sim,1,masculino,38.00,1,0,19943,S
409,sim,1,feminino,58.00,0,1,PC 17582,S


## Preenchimento de valores nulos


### Buscando colunas com valores nulos

In [8]:
print('---df_train---')
for coluna in df_train:
  print(coluna + ':', df_train[coluna].isnull().values.sum())

print('\n---df_test---')
for coluna in df_test:
  print(coluna + ':', df_test[coluna].isnull().values.sum())

---df_train---
Sobreviveu: 0
ClassePassageiro: 0
Sexo: 0
Idade: 142
Parentes a Bordo: 0
Pais / Crianças a Bordo: 0
Ticket: 0
Embarque: 2

---df_test---
ClassePassageiro: 0
Sexo: 0
Idade: 35
Parentes a Bordo: 0
Pais / Crianças a Bordo: 0
Ticket: 0
Embarque: 0


### Preenchendo Idade

Idade será preenchida com a média das idades (em inteiro).

In [9]:
df_aux1 = df_train[['Idade']]
df_aux2 = df_test[['Idade']]

df_train['Idade'] = df_aux1.fillna(int(df_train['Idade'].mean()))
df_test['Idade'] = df_aux2.fillna(int(df_test['Idade'].mean()))

# Checa se restam valores nan
print(df_train['Idade'].isnull().values.any())
print(df_test['Idade'].isnull().values.any())

df_train.head()

False
False


,Sobreviveu,ClassePassageiro,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Embarque
0,nao,3,feminino,20.0,1,0,4136,S
1,nao,2,masculino,34.0,1,0,28664,S
2,nao,3,masculino,29.0,8,2,CA. 2343,S
3,sim,1,feminino,38.0,0,0,PC 17757,C
4,nao,3,masculino,61.0,0,0,345364,S


### Preenchendo Embarque

Serão preenchidas com outra classe: 'N'.

In [10]:
df_aux1 = df_train[['Embarque']]
df_aux2 = df_test[['Embarque']]

df_train['Embarque'] = df_aux1.fillna('N')
df_test['Embarque'] = df_aux2.fillna('N')

# Checa se restam valores nan
print(df_train['Embarque'].isnull().values.any())
print(df_test['Embarque'].isnull().values.any())

df_train.head()

False
False


,Sobreviveu,ClassePassageiro,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Embarque
0,nao,3,feminino,20.0,1,0,4136,S
1,nao,2,masculino,34.0,1,0,28664,S
2,nao,3,masculino,29.0,8,2,CA. 2343,S
3,sim,1,feminino,38.0,0,0,PC 17757,C
4,nao,3,masculino,61.0,0,0,345364,S


### Tratamento especial: Ticket
Considera-se somente a parte decimal do ticket, e faz uma nova coluna com a parte não decimal.

In [11]:
listaTicketsTrain = df_train['Ticket'].copy()
listaTicketsTrainClasses = df_train['Ticket'].copy()

listaTicketsTest = df_test['Ticket'].copy()
listaTicketsTestClasses = df_test['Ticket'].copy()

for i, val in enumerate(listaTicketsTrain):
  aux = val.split()

  valordec = 0
  valorstr = 'N'

  for elem in aux:
    if elem.isdecimal():
      valordec = int(elem)
    else:
      valorstr = valorstr + elem

  listaTicketsTrain[i] = valordec
  listaTicketsTrainClasses[i] = valorstr

for i, val in enumerate(listaTicketsTest):
  aux = val.split()

  valordec = 0
  valorstr = 'N'

  for elem in aux:
    if elem.isdecimal():
      valordec = int(elem)
    else:
      valorstr = valorstr + elem

  listaTicketsTest[i] = valordec
  listaTicketsTestClasses[i] = valorstr

df_train['Ticket'] = listaTicketsTrain
df_test['Ticket'] = listaTicketsTest

# Fazer nova coluna com 'TicketClasse' abaixa score
'''
df_train['TicketClasse'] = listaTicketsTrainClasses
df_test['TicketClasse'] = listaTicketsTestClasses
'''

df_train.head()

,Sobreviveu,ClassePassageiro,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Embarque
0,nao,3,feminino,20.0,1,0,4136,S
1,nao,2,masculino,34.0,1,0,28664,S
2,nao,3,masculino,29.0,8,2,2343,S
3,sim,1,feminino,38.0,0,0,17757,C
4,nao,3,masculino,61.0,0,0,345364,S


## Binarização de Atributos Categóricos

### Transformações

In [12]:
# Transformação de atributo nominal em boolean (sobreviveu)
dictaux = {"nao": False, "sim": True}
df_train['Sobreviveu'] = df_train['Sobreviveu'].map(dictaux)

# Categorização do tipo [0:1] (Sexo)
dictaux = {"masculino": 0, "feminino": 1}
df_train['Sexo'] = df_train['Sexo'].map(dictaux)
df_test['Sexo'] = df_test['Sexo'].map(dictaux)

# Transformação de atributo nominal em numérico (Embarque)
dictaux = {"S": 0, "C": 1, "Q": 2, "N": 3}
df_train['Embarque'] = df_train['Embarque'].map(dictaux)
df_test['Embarque'] = df_test['Embarque'].map(dictaux)


# Fazer nova coluna com 'TicketClasse' abaixa score
'''
# Transformação de atributo nominal em numérico (TicketClasse)
dictaux1 = {}
dictaux2 = {}

for i, val in enumerate(df_train['TicketClasse'].unique()):
  dictaux1[val] = i

for i, val in enumerate(df_test['TicketClasse'].unique()):
  dictaux2[val] = i

df_train['TicketClasse'] = df_train['TicketClasse'].map(dictaux1)
df_test['TicketClasse'] = df_test['TicketClasse'].map(dictaux2)
'''

df_train.head()

,Sobreviveu,ClassePassageiro,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Embarque
0,False,3,1,20.0,1,0,4136,0
1,False,2,0,34.0,1,0,28664,0
2,False,3,0,29.0,8,2,2343,0
3,True,1,1,38.0,0,0,17757,1
4,False,3,0,61.0,0,0,345364,0


### Preparação dos dados - Dataframes X, y e X_pred

In [13]:
# Criando X e y a partir de df_train
X = df_train.drop(columns = 'Sobreviveu')
y = df_train['Sobreviveu']

# Criação de variável para predição
X_pred = df_test.copy()

In [14]:
print(X_pred.shape)
print(X.shape)

X.head()

(179, 7)
(712, 7)


,ClassePassageiro,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Embarque
0,3,1,20.0,1,0,4136,0
1,2,0,34.0,1,0,28664,0
2,3,0,29.0,8,2,2343,0
3,1,1,38.0,0,0,17757,1
4,3,0,61.0,0,0,345364,0


### Binarização

In [15]:
# Função que binariza um dataset a partir de uma lista de possíveis colunas
def binariza(X_atual, listabin):
  aux = X_atual.copy()

  for feature in X_atual.columns:
    if feature in listabin:
      dummies = pd.get_dummies(aux[feature], prefix=feature)
      aux = pd.concat([aux, dummies], axis=1)
      aux = aux.drop(columns=[feature])

  return aux

In [16]:
# Garante que dois datasets tenham as mesmas colunas
def normalizacoluna(X1, X2):
  for coluna in X1:
    if coluna not in X2.columns:
      X2[coluna] = np.zeros((X2.shape[0],), dtype=int)

  for coluna in X2:
    if coluna not in X1.columns:
      X2[coluna] = np.zeros((X1.shape[0],), dtype=int)

bin1_x: Binariza TicketClasse e Embarque

In [17]:
listabin = ['TicketClasse', 'Embarque']

bin1_X = binariza(X, listabin)
bin1_Xpred = binariza(X_pred, listabin)

normalizacoluna(bin1_X, bin1_Xpred)

print(bin1_X.shape)
print(bin1_Xpred.shape)

(712, 10)
(179, 10)


bin2_X: Binariza TicketClasse, Embarque e  	ClassePassageiro


In [18]:
listabin = ['TicketClasse', 'Embarque', 'ClassePassageiro']

bin2_X = binariza(X, listabin)
bin2_Xpred = binariza(X_pred, listabin)

normalizacoluna(bin2_X, bin2_Xpred)

print(bin2_X.shape)
print(bin2_Xpred.shape)

(712, 12)
(179, 12)


bin3_X: Binariza TicketClasse, Embarque, ClassePassageiro e Pais / Crianças a Bordo


In [19]:
listabin = ['TicketClasse', 'Embarque', 'ClassePassageiro', 'Pais / Crianças a Bordo']

bin3_X = binariza(X, listabin)
bin3_Xpred = binariza(X_pred, listabin)

normalizacoluna(bin3_X, bin3_Xpred)

print(bin3_X.shape)
print(bin3_Xpred.shape)

(712, 18)
(179, 18)


# Algoritmos de ML

### Preparação dos dados - Conjuntos de teste
Usamos técnica holdout para testar os algoritmos no final.

In [20]:
X_train0, X_test0, y_train0, y_test0 = train_test_split(X, y, test_size=0.2, random_state=rnd)
X_train1, X_test1, y_train1, y_test1 = train_test_split(bin1_X, y, test_size=0.2, random_state=rnd)
X_train2, X_test2, y_train2, y_test2 = train_test_split(bin2_X, y, test_size=0.2, random_state=rnd)
X_train3, X_test3, y_train3, y_test3 = train_test_split(bin3_X, y, test_size=0.2, random_state=rnd)

### Funções usadas

In [21]:
# Obtém resultados a partir de clf.cv_results_
def get_results(res):
  cols = list(res.keys())
  cols = ["rank_test_score"] + [c for c in cols if c.startswith("param_")] + [
          "mean_fit_time",
          "std_fit_time",
          "mean_test_score",
           "std_test_score" 
        ]
  return pd.DataFrame(res)[cols].sort_values("mean_test_score", ascending=False)

Dicionário para armazenamento das melhores scores.

In [22]:
listascores = {}

## Algoritmo SVM - SVC

### Grid Search

In [23]:
def gridSearchSVM(X, y):
  C_lst = np.logspace(-3, 4, 10)
  gamma_lst = np.logspace(-3, 3, 10).tolist()
  gamma_lst.append('scale')
  gamma_lst.append('auto')
  coef_lst = np.logspace(0, 2, 3)

  grid_parametros = [
    {
        'kernel': ['linear'],
        'C': C_lst,
        'gamma': gamma_lst,
        'max_iter': [100],
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['poly'],
        'degree': [2, 3, 4],
        'coef0': coef_lst,
        'C': C_lst,
        'gamma': gamma_lst,
        'max_iter': [100],
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['sigmoid'],
        'coef0': coef_lst,
        'C': C_lst,
        'gamma': gamma_lst,
        'max_iter': [100],
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['rbf'],
        'C': C_lst,
        'gamma': gamma_lst,
        'class_weight': [None, 'balanced']
    }]

  svc = SVC()
  clf = GridSearchCV(estimator=svc, param_grid=grid_parametros, n_jobs=-1)

  scaler = MinMaxScaler()
  clf = Pipeline([("scaler", scaler),
                  ("clf", clf)])
  
  clf.fit(X, y)

  return pd.DataFrame.from_dict(clf['clf'].cv_results_), clf, clf['clf'].best_score_

Sem binarização

In [24]:
aux = gridSearchSVM(X_train0, y_train0)
listascores['SVM - sembin - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_gamma,param_kernel,param_max_iter,param_coef0,param_degree,mean_fit_time,std_fit_time,mean_test_score,std_test_score
3299,1,278.25594,None,auto,rbf,NaN,NaN,NaN,0.023432,0.002333,0.797873,0.026740
3315,2,1668.100537,None,0.1,rbf,NaN,NaN,NaN,0.051674,0.009013,0.796103,0.023418
3323,3,1668.100537,None,auto,rbf,NaN,NaN,NaN,0.063033,0.017092,0.794349,0.019197
3291,4,278.25594,None,0.1,rbf,NaN,NaN,NaN,0.024990,0.010570,0.792610,0.031726
3338,4,10000.0,None,0.021544,rbf,NaN,NaN,NaN,0.067625,0.010416,0.792610,0.028127
...,...,...,...,...,...,...,...,...,...,...,...,...
2165,3356,1668.100537,balanced,2.154435,poly,100,100.0,3,0.006056,0.003751,0.342618,0.035646
2625,3357,0.215443,None,1000.0,sigmoid,100,1.0,NaN,0.007723,0.003231,0.341018,0.032848
2769,3358,7.742637,None,1000.0,sigmoid,100,1.0,NaN,0.006372,0.000918,0.341003,0.026400
1733,3359,46.415888,balanced,2.154435,poly,100,100.0,3,0.004772,0.001675,0.323319,0.054731


bin1_x

In [25]:
aux = gridSearchSVM(X_train1, y_train1)
listascores['SVM - bin1_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_gamma,param_kernel,param_max_iter,param_coef0,param_degree,mean_fit_time,std_fit_time,mean_test_score,std_test_score
3338,1,10000.0,None,0.021544,rbf,NaN,NaN,NaN,0.075638,0.027725,0.812017,0.020719
3291,2,278.25594,None,0.1,rbf,NaN,NaN,NaN,0.021278,0.002980,0.811986,0.012661
3299,2,278.25594,None,auto,rbf,NaN,NaN,NaN,0.021648,0.002898,0.811986,0.012661
3250,2,7.742637,None,scale,rbf,NaN,NaN,NaN,0.013644,0.000907,0.811986,0.012661
3315,5,1668.100537,None,0.1,rbf,NaN,NaN,NaN,0.053183,0.010177,0.808508,0.025233
...,...,...,...,...,...,...,...,...,...,...,...,...
2780,3355,7.742637,None,215.443469,sigmoid,100,10.0,NaN,0.005993,0.000091,0.333985,0.027484
2636,3357,0.215443,None,215.443469,sigmoid,100,10.0,NaN,0.010527,0.004097,0.333970,0.020111
2297,3358,10000.0,balanced,2.154435,poly,100,1.0,2,0.004088,0.000213,0.333861,0.054310
3068,3359,10000.0,None,215.443469,sigmoid,100,10.0,NaN,0.005947,0.000078,0.332200,0.021650


bin2_x

In [26]:
aux = gridSearchSVM(X_train2, y_train2)
listascores['SVM - bin2_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_gamma,param_kernel,param_max_iter,param_coef0,param_degree,mean_fit_time,std_fit_time,mean_test_score,std_test_score
3257,1,7.742637,balanced,2.154435,rbf,NaN,NaN,NaN,0.018951,0.002544,0.804937,0.044794
3269,2,46.415888,None,2.154435,rbf,NaN,NaN,NaN,0.026772,0.007065,0.803229,0.035484
3310,3,278.25594,balanced,scale,rbf,NaN,NaN,NaN,0.041859,0.007890,0.803183,0.031556
3274,4,46.415888,None,scale,rbf,NaN,NaN,NaN,0.017087,0.002546,0.801475,0.038388
3323,4,1668.100537,None,auto,rbf,NaN,NaN,NaN,0.044741,0.006641,0.801475,0.032291
...,...,...,...,...,...,...,...,...,...,...,...,...
178,3349,278.25594,None,scale,linear,100,NaN,NaN,0.005146,0.002657,0.328443,0.071430
175,3349,278.25594,None,46.415888,linear,100,NaN,NaN,0.003297,0.000098,0.328443,0.071430
176,3349,278.25594,None,215.443469,linear,100,NaN,NaN,0.003350,0.000119,0.328443,0.071430
179,3349,278.25594,None,auto,linear,100,NaN,NaN,0.005807,0.003226,0.328443,0.071430


bin3_x

In [27]:
aux = gridSearchSVM(X_train3, y_train3)
listascores['SVM - bin3_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_gamma,param_kernel,param_max_iter,param_coef0,param_degree,mean_fit_time,std_fit_time,mean_test_score,std_test_score
3244,1,7.742637,None,0.464159,rbf,NaN,NaN,NaN,0.020675,0.004309,0.782161,0.035712
3267,2,46.415888,None,0.1,rbf,NaN,NaN,NaN,0.021416,0.002884,0.782146,0.037035
3337,3,10000.0,None,0.004642,rbf,NaN,NaN,NaN,0.063241,0.017288,0.780391,0.036434
3315,3,1668.100537,None,0.1,rbf,NaN,NaN,NaN,0.117621,0.017811,0.780391,0.031932
3250,3,7.742637,None,scale,rbf,NaN,NaN,NaN,0.016246,0.000399,0.780391,0.033346
...,...,...,...,...,...,...,...,...,...,...,...,...
2070,3355,1668.100537,None,10.0,poly,100,100.0,4,0.003225,0.000412,0.372628,0.058107
2879,3357,46.415888,balanced,auto,sigmoid,100,1.0,NaN,0.012463,0.004935,0.372551,0.016458
1068,3358,0.215443,balanced,0.001,poly,100,100.0,2,0.011734,0.003419,0.372504,0.022790
1637,3359,46.415888,None,2.154435,poly,100,100.0,4,0.007325,0.003827,0.367210,0.042734


### Randomized Search

In [28]:
def randomizedSearchSVM(X, y):
  C_lst = loguniform(10**-3, 10**4)
  gamma_lst = loguniform(10**-3, 10**3)
  coef_lst = loguniform(10**0, 10**2)

  grid_parametros = [
    {
        'kernel': ['linear'],
        'C': C_lst,
        'gamma': gamma_lst,
        'max_iter': [100],
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['poly'],
        'degree': [2, 3, 4],
        'coef0': coef_lst,
        'C': C_lst,
        'gamma': gamma_lst,
        'max_iter': [100],
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['sigmoid'],
        'degree': [2, 3, 4],
        'coef0': coef_lst,
        'C': C_lst,
        'gamma': gamma_lst,
        'max_iter': [100],
        'class_weight': [None, 'balanced']
    },
    {
        'kernel': ['rbf'],
        'C': C_lst,
        'gamma': gamma_lst,
        'class_weight': [None, 'balanced']
    }]

  svc = SVC()
  clf = RandomizedSearchCV(estimator=svc, param_distributions=grid_parametros, n_jobs=-1, n_iter = 500)

  scaler = MinMaxScaler()
  clf = Pipeline([("scaler", scaler),
                  ("clf", clf)])
  
  clf.fit(X, y)

  return pd.DataFrame.from_dict(clf['clf'].cv_results_), clf, clf['clf'].best_score_

Sem binarização

In [29]:
aux = randomizedSearchSVM(X_train0, y_train0)
listascores['SVM - sembin - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_gamma,param_kernel,param_max_iter,param_coef0,param_degree,mean_fit_time,std_fit_time,mean_test_score,std_test_score
191,1,8.078422,None,0.977399,rbf,NaN,NaN,NaN,0.016261,0.005496,0.796119,0.028064
156,2,348.828728,None,0.253454,rbf,NaN,NaN,NaN,0.031105,0.006387,0.796103,0.023418
8,3,1.935924,None,3.709052,rbf,NaN,NaN,NaN,0.014284,0.003601,0.794364,0.022703
208,4,5.440427,None,1.548855,rbf,NaN,NaN,NaN,0.013281,0.000625,0.792594,0.026566
210,5,2.781287,None,8.213707,rbf,NaN,NaN,NaN,0.017433,0.006306,0.790840,0.013228
...,...,...,...,...,...,...,...,...,...,...,...,...
152,496,0.071733,balanced,905.719138,sigmoid,100,3.66847,2,0.007260,0.006509,0.369104,0.025927
306,497,0.009973,None,455.850907,sigmoid,100,13.146983,4,0.007178,0.002690,0.369027,0.028295
150,498,344.798745,balanced,388.81718,sigmoid,100,6.900955,2,0.004035,0.000117,0.367350,0.027915
114,499,896.721788,None,421.558483,sigmoid,100,12.199146,4,0.007930,0.004144,0.365518,0.024655


bin1_x

In [30]:
aux = randomizedSearchSVM(X_train1, y_train1)
listascores['SVM - bin1_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_gamma,param_kernel,param_coef0,param_degree,param_max_iter,mean_fit_time,std_fit_time,mean_test_score,std_test_score
307,1,46.338652,None,0.356456,rbf,NaN,NaN,NaN,0.016106,0.001220,0.815510,0.015226
112,2,575.600354,None,0.08279,rbf,NaN,NaN,NaN,0.028390,0.003094,0.812001,0.015759
473,3,1415.733979,None,0.10366,rbf,NaN,NaN,NaN,0.048180,0.005588,0.810262,0.024130
218,4,4499.782906,None,0.030108,rbf,NaN,NaN,NaN,0.055076,0.015134,0.810231,0.012620
220,5,0.6391,None,3.146339,rbf,NaN,NaN,NaN,0.012199,0.000802,0.801459,0.022196
...,...,...,...,...,...,...,...,...,...,...,...,...
326,496,43.70483,balanced,25.115015,sigmoid,2.433079,4,100,0.005191,0.000197,0.365533,0.011211
12,497,3312.529376,balanced,651.171054,sigmoid,2.165442,2,100,0.004399,0.000167,0.360301,0.021072
58,498,20.257666,None,360.072325,sigmoid,3.467524,3,100,0.006005,0.000060,0.358578,0.018153
480,499,751.843365,None,0.18226,poly,1.040573,4,100,0.004603,0.000405,0.342726,0.011643


bin2_x

In [31]:
aux = randomizedSearchSVM(X_train2, y_train2)
listascores['SVM - bin2_x - RandomizedSearch'] = (aux[1], aux[2])
 
get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_coef0,param_degree,param_gamma,param_kernel,param_max_iter,mean_fit_time,std_fit_time,mean_test_score,std_test_score
277,1,304.099673,balanced,NaN,NaN,0.507313,rbf,NaN,0.046911,0.006764,0.803183,0.031556
358,2,859.836241,None,NaN,NaN,0.10347,rbf,NaN,0.033364,0.008877,0.801490,0.035854
459,3,30.782118,None,NaN,NaN,0.509978,rbf,NaN,0.015657,0.001946,0.801459,0.027743
10,4,2721.240535,None,NaN,NaN,0.120798,rbf,NaN,0.096944,0.027128,0.797966,0.033386
50,5,0.849804,None,NaN,NaN,1.950793,rbf,NaN,0.019400,0.003690,0.797951,0.031549
...,...,...,...,...,...,...,...,...,...,...,...,...
463,496,1841.199096,None,1.717092,2,1.770505,poly,100,0.003845,0.000255,0.377690,0.061005
214,497,0.120866,balanced,2.039953,3,12.177687,poly,100,0.005571,0.002108,0.376122,0.053768
173,498,474.716105,None,1.018245,3,3.772948,poly,100,0.003037,0.000416,0.375904,0.064706
453,499,0.117056,None,70.682857,3,38.138704,poly,100,0.004183,0.001072,0.361916,0.036396


bin3_x

In [32]:
aux = randomizedSearchSVM(X_train3, y_train3)
listascores['SVM - bin3_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_C,param_class_weight,param_coef0,param_degree,param_gamma,param_kernel,param_max_iter,mean_fit_time,std_fit_time,mean_test_score,std_test_score
466,1,2.502479,None,NaN,NaN,0.884937,rbf,NaN,0.016995,0.000490,0.789179,0.040072
118,2,5.111702,None,NaN,NaN,0.268788,rbf,NaN,0.016018,0.000611,0.785623,0.040445
456,3,88.656908,None,NaN,NaN,0.0574,rbf,NaN,0.020526,0.001578,0.783900,0.039541
378,4,886.333877,None,NaN,NaN,0.013986,rbf,NaN,0.033267,0.009990,0.783884,0.035063
128,5,84.432117,None,NaN,NaN,0.624661,rbf,NaN,0.040908,0.007443,0.782161,0.030607
...,...,...,...,...,...,...,...,...,...,...,...,...
134,397,1.943152,balanced,69.460541,2,0.001034,poly,100,0.005036,0.000121,0.383124,0.003636
413,497,0.042353,balanced,6.553707,4,5.486586,sigmoid,100,0.007645,0.000905,0.377861,0.009701
366,497,2888.451333,balanced,14.377609,2,7.680049,poly,100,0.003317,0.000383,0.377861,0.046766
312,499,1113.857801,balanced,4.261726,2,1.209425,sigmoid,100,0.007696,0.000192,0.374383,0.026962


## KNN

### Grid Search

In [33]:
def gridSearchKNN(X, y):
  neigh_lst = [10, 20, 30, 40]
  weight_lst = ['uniform', 'distance']
  alg_lst = ['ball_tree', 'kd_tree', 'brute']
  leaf_lst = [10, 20, 30, 40, 50]
  p_lst = [1, 2, 3]

  grid_parametros = [
    {
        'n_neighbors': neigh_lst,
        'weights': weight_lst,
        'algorithm': alg_lst,
        'leaf_size': leaf_lst,
        'p': p_lst
    }]

  knn = KNeighborsClassifier()
  clf = GridSearchCV(estimator=knn, param_grid=grid_parametros, n_jobs=-1)

  scaler = MinMaxScaler()
  clf = Pipeline([("scaler", scaler),
                  ("clf", clf)])
  
  clf.fit(X, y)

  return pd.DataFrame.from_dict(clf['clf'].cv_results_), clf, clf['clf'].best_score_

Sem binarização

In [34]:
aux = gridSearchKNN(X_train0, y_train0)
listascores['KNN - sembin - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,mean_fit_time,std_fit_time,mean_test_score,std_test_score
359,1,brute,50,40,3,distance,0.001171,0.000057,0.789101,0.018415
191,1,kd_tree,30,40,3,distance,0.001745,0.000087,0.789101,0.018415
71,1,ball_tree,30,40,3,distance,0.003471,0.001507,0.789101,0.018415
47,1,ball_tree,20,40,3,distance,0.003639,0.000973,0.789101,0.018415
95,1,ball_tree,40,40,3,distance,0.002783,0.000091,0.789101,0.018415
...,...,...,...,...,...,...,...,...,...,...
320,346,brute,40,20,2,uniform,0.001061,0.000084,0.766232,0.017519
80,346,ball_tree,40,20,2,uniform,0.001535,0.000109,0.766232,0.017519
8,346,ball_tree,10,20,2,uniform,0.001660,0.000061,0.766232,0.017519
296,346,brute,30,20,2,uniform,0.001040,0.000033,0.766232,0.017519


bin1_x

In [35]:
aux = gridSearchKNN(X_train1, y_train1)
listascores['KNN - bin1_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,mean_fit_time,std_fit_time,mean_test_score,std_test_score
0,1,ball_tree,10,10,1,uniform,0.004347,0.003105,0.796181,0.018271
216,1,kd_tree,50,10,1,uniform,0.001832,0.000058,0.796181,0.018271
24,1,ball_tree,20,10,1,uniform,0.001676,0.000084,0.796181,0.018271
288,1,brute,30,10,1,uniform,0.001253,0.000057,0.796181,0.018271
48,1,ball_tree,30,10,1,uniform,0.001601,0.000088,0.796181,0.018271
...,...,...,...,...,...,...,...,...,...,...
317,346,brute,40,10,3,distance,0.001239,0.000049,0.773312,0.031401
29,346,ball_tree,20,10,3,distance,0.003723,0.000086,0.773312,0.031401
293,346,brute,30,10,3,distance,0.001605,0.000860,0.773312,0.031401
197,346,kd_tree,40,10,3,distance,0.001854,0.000078,0.773312,0.031401


bin2_x

In [36]:
aux = gridSearchKNN(X_train2, y_train2)
listascores['KNN - bin2_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,mean_fit_time,std_fit_time,mean_test_score,std_test_score
349,1,brute,50,30,1,distance,0.001275,0.000437,0.785592,0.037436
253,1,brute,10,30,1,distance,0.001141,0.000142,0.785592,0.037436
229,1,kd_tree,50,30,1,distance,0.001726,0.000095,0.785592,0.037436
181,1,kd_tree,30,30,1,distance,0.001715,0.000100,0.785592,0.037436
301,1,brute,30,30,1,distance,0.001219,0.000132,0.785592,0.037436
...,...,...,...,...,...,...,...,...,...,...
184,331,kd_tree,30,30,3,uniform,0.001863,0.000026,0.761015,0.028252
328,331,brute,40,30,3,uniform,0.001175,0.000113,0.761015,0.028252
182,331,kd_tree,30,30,2,uniform,0.001957,0.000396,0.761015,0.028252
326,331,brute,40,30,2,uniform,0.001127,0.000081,0.761015,0.028252


bin3_x

In [37]:
aux = gridSearchKNN(X_train3, y_train3)
listascores['KNN - bin3_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_algorithm,param_leaf_size,param_n_neighbors,param_p,param_weights,mean_fit_time,std_fit_time,mean_test_score,std_test_score
226,1,kd_tree,50,20,3,uniform,0.002430,0.000610,0.778637,0.030635
250,1,brute,10,20,3,uniform,0.001266,0.000133,0.778637,0.030635
178,1,kd_tree,30,20,3,uniform,0.002128,0.000048,0.778637,0.030635
82,1,ball_tree,40,20,3,uniform,0.004511,0.000116,0.778637,0.030635
322,1,brute,40,20,3,uniform,0.001303,0.000152,0.778637,0.030635
...,...,...,...,...,...,...,...,...,...,...
212,331,kd_tree,40,40,2,uniform,0.002020,0.000041,0.723987,0.032575
334,331,brute,40,40,3,uniform,0.001189,0.000093,0.723987,0.032575
68,331,ball_tree,30,40,2,uniform,0.001822,0.000065,0.723987,0.032575
70,331,ball_tree,30,40,3,uniform,0.004487,0.000119,0.723987,0.032575


### Randomized Search

In [38]:
def randomizedSearchKNN(X, y):
  neigh_lst = [10, 20, 30, 40]
  weight_lst = ['uniform', 'distance']
  alg_lst = ['ball_tree', 'kd_tree', 'brute']
  leaf_lst = [10, 20, 30, 40, 50]
  p_lst = [1, 2, 3]

  grid_parametros = [
    {
        'n_neighbors': neigh_lst,
        'weights': weight_lst,
        'algorithm': alg_lst,
        'leaf_size': leaf_lst,
        'p': p_lst
    }]

  knn = KNeighborsClassifier()
  clf = RandomizedSearchCV(estimator=knn, param_distributions=grid_parametros, n_jobs=-1, n_iter=200)

  scaler = MinMaxScaler()
  clf = Pipeline([("scaler", scaler),
                  ("clf", clf)])
  
  clf.fit(X, y)

  return pd.DataFrame.from_dict(clf['clf'].cv_results_), clf, clf['clf'].best_score_

Sem binarização

In [39]:
aux = randomizedSearchKNN(X_train0, y_train0)
listascores['KNN - sembin - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_weights,param_p,param_n_neighbors,param_leaf_size,param_algorithm,mean_fit_time,std_fit_time,mean_test_score,std_test_score
57,1,distance,3,40,30,ball_tree,0.002940,0.000137,0.789101,0.018415
126,1,distance,3,40,50,ball_tree,0.002881,0.000182,0.789101,0.018415
193,1,distance,3,40,10,brute,0.001288,0.000052,0.789101,0.018415
73,1,distance,3,40,40,brute,0.003154,0.003797,0.789101,0.018415
20,1,distance,3,40,50,kd_tree,0.001701,0.000051,0.789101,0.018415
...,...,...,...,...,...,...,...,...,...,...
36,195,uniform,2,20,40,brute,0.001232,0.000046,0.766232,0.017519
89,195,uniform,2,20,20,kd_tree,0.001712,0.000039,0.766232,0.017519
192,195,uniform,2,20,50,ball_tree,0.001717,0.000093,0.766232,0.017519
9,195,uniform,2,20,30,kd_tree,0.001709,0.000074,0.766232,0.017519


bin1_x

In [40]:
aux = randomizedSearchKNN(X_train1, y_train1)
listascores['KNN - bin1_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_weights,param_p,param_n_neighbors,param_leaf_size,param_algorithm,mean_fit_time,std_fit_time,mean_test_score,std_test_score
9,1,uniform,1,10,30,kd_tree,0.001880,0.000041,0.796181,0.018271
167,1,uniform,1,10,10,ball_tree,0.001686,0.000127,0.796181,0.018271
99,1,uniform,1,10,30,brute,0.001070,0.000062,0.796181,0.018271
34,1,uniform,1,10,40,brute,0.002532,0.002485,0.796181,0.018271
178,1,uniform,1,10,40,kd_tree,0.001899,0.000551,0.796181,0.018271
...,...,...,...,...,...,...,...,...,...,...
27,193,distance,3,10,10,brute,0.001265,0.000047,0.773312,0.031401
74,193,distance,3,10,30,brute,0.001193,0.000081,0.773312,0.031401
14,193,distance,3,10,10,ball_tree,0.005599,0.002884,0.773312,0.031401
196,193,distance,3,10,10,kd_tree,0.002018,0.000090,0.773312,0.031401


bin2_x

In [41]:
aux = randomizedSearchKNN(X_train2, y_train2)
listascores['KNN - bin2_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_weights,param_p,param_n_neighbors,param_leaf_size,param_algorithm,mean_fit_time,std_fit_time,mean_test_score,std_test_score
147,1,distance,1,30,30,kd_tree,0.001800,0.000113,0.785592,0.037436
167,1,distance,1,30,30,ball_tree,0.001638,0.000113,0.785592,0.037436
56,1,distance,1,30,50,kd_tree,0.002203,0.000871,0.785592,0.037436
191,1,distance,1,30,40,ball_tree,0.001710,0.000084,0.785592,0.037436
54,1,distance,1,30,10,ball_tree,0.001600,0.000030,0.785592,0.037436
...,...,...,...,...,...,...,...,...,...,...
105,181,uniform,3,30,40,kd_tree,0.003412,0.001974,0.761015,0.028252
77,181,uniform,2,30,40,kd_tree,0.001946,0.000169,0.761015,0.028252
103,181,uniform,3,30,30,kd_tree,0.001886,0.000037,0.761015,0.028252
135,181,uniform,3,30,40,brute,0.001272,0.000050,0.761015,0.028252


bin3_x

In [42]:
aux = randomizedSearchKNN(X_train3, y_train3)
listascores['KNN - bin3_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_weights,param_p,param_n_neighbors,param_leaf_size,param_algorithm,mean_fit_time,std_fit_time,mean_test_score,std_test_score
124,1,uniform,3,20,40,kd_tree,0.003124,0.002133,0.778637,0.030635
168,1,uniform,3,20,30,ball_tree,0.004365,0.000197,0.778637,0.030635
148,1,uniform,3,20,20,ball_tree,0.005062,0.000129,0.778637,0.030635
81,1,uniform,3,20,10,kd_tree,0.002463,0.000077,0.778637,0.030635
80,1,uniform,3,20,20,kd_tree,0.002263,0.000072,0.778637,0.030635
...,...,...,...,...,...,...,...,...,...,...
73,189,uniform,2,40,10,kd_tree,0.002351,0.000017,0.723987,0.032575
37,189,uniform,2,40,50,brute,0.001521,0.000302,0.723987,0.032575
69,189,uniform,3,40,30,ball_tree,0.005769,0.002087,0.723987,0.032575
54,189,uniform,2,40,20,kd_tree,0.002228,0.000020,0.723987,0.032575


## Árvore de Decisão

### Grid Search

In [43]:
def gridSearchTree(X, y):
  crit_lst = ['gini', 'entropy']
  split_lst = ['best', 'random']
  minsplit_lst = [2, 0.1, 0.2, 0.3]
  minsamp_lst = [1, 0.1, 0.2, 0.3]
  classweight_lst = [None, 'balanced']

  grid_parametros = [
    {
        'criterion': crit_lst,
        'splitter': split_lst,
        'min_samples_split': minsplit_lst,
        'min_samples_leaf': minsamp_lst,
        'class_weight': classweight_lst
    }]

  tree = DecisionTreeClassifier()
  clf = GridSearchCV(estimator=tree, param_grid=grid_parametros, n_jobs=-1)

  scaler = MinMaxScaler()
  clf = Pipeline([("scaler", scaler),
                  ("clf", clf)])
  
  clf.fit(X, y)

  return pd.DataFrame.from_dict(clf['clf'].cv_results_), clf, clf['clf'].best_score_

Sem binarização

In [44]:
aux = gridSearchTree(X_train0, y_train0)
listascores['Tree - sembin - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_class_weight,param_criterion,param_min_samples_leaf,param_min_samples_split,param_splitter,mean_fit_time,std_fit_time,mean_test_score,std_test_score
3,1,None,gini,1,0.1,random,0.002397,0.001971,0.789023,0.043084
35,2,None,entropy,1,0.1,random,0.002072,0.001036,0.775019,0.033267
34,3,None,entropy,1,0.1,best,0.003818,0.002000,0.775019,0.009541
98,4,balanced,entropy,1,0.1,best,0.003384,0.000073,0.769756,0.030191
45,5,None,entropy,0.1,0.2,random,0.002483,0.002072,0.766279,0.028433
...,...,...,...,...,...,...,...,...,...,...
96,124,balanced,entropy,1,2,best,0.004562,0.000170,0.725834,0.017870
97,125,balanced,entropy,1,2,random,0.002661,0.000084,0.715231,0.020600
68,126,balanced,gini,1,0.2,best,0.003682,0.002759,0.715200,0.038648
64,127,balanced,gini,1,2,best,0.006756,0.003208,0.713538,0.022510


bin1_x

In [45]:
aux = gridSearchTree(X_train1, y_train1)
listascores['Tree - bin1_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_class_weight,param_criterion,param_min_samples_leaf,param_min_samples_split,param_splitter,mean_fit_time,std_fit_time,mean_test_score,std_test_score
35,1,None,entropy,1,0.1,random,0.001447,0.000046,0.810247,0.036816
3,2,None,gini,1,0.1,random,0.001481,0.000096,0.796181,0.024084
2,3,None,gini,1,0.1,best,0.002077,0.000103,0.783838,0.025719
43,4,None,entropy,0.1,0.1,random,0.001241,0.000010,0.778590,0.020735
11,5,None,gini,0.1,0.1,random,0.001289,0.000016,0.776820,0.019473
...,...,...,...,...,...,...,...,...,...,...
76,118,balanced,gini,0.1,0.2,best,0.004254,0.004058,0.720602,0.032684
78,118,balanced,gini,0.1,0.3,best,0.002049,0.000097,0.720602,0.032684
104,118,balanced,entropy,0.1,2,best,0.002182,0.000054,0.720602,0.032684
102,127,balanced,entropy,1,0.3,best,0.004152,0.002961,0.720509,0.036138


bin2_x

In [46]:
aux = gridSearchTree(X_train2, y_train2)
listascores['Tree - bin2_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_class_weight,param_criterion,param_min_samples_leaf,param_min_samples_split,param_splitter,mean_fit_time,std_fit_time,mean_test_score,std_test_score
35,1,None,entropy,1,0.1,random,0.003184,0.003284,0.771588,0.050310
2,2,None,gini,1,0.1,best,0.002669,0.001125,0.771526,0.025436
3,3,None,gini,1,0.1,random,0.001436,0.000060,0.769834,0.046943
34,4,None,entropy,1,0.1,best,0.004332,0.002780,0.762754,0.050174
67,5,balanced,gini,1,0.1,random,0.001908,0.000038,0.760984,0.045199
...,...,...,...,...,...,...,...,...,...,...
106,116,balanced,entropy,0.1,0.1,best,0.002278,0.000062,0.729390,0.032695
96,125,balanced,entropy,1,2,best,0.004663,0.000317,0.727558,0.026962
33,126,None,entropy,1,2,random,0.001952,0.000031,0.722264,0.032677
101,127,balanced,entropy,1,0.2,random,0.002480,0.000697,0.717016,0.033736


bin3_x

In [47]:
aux = gridSearchTree(X_train3, y_train3)
listascores['Tree - bin3_x - GridSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_class_weight,param_criterion,param_min_samples_leaf,param_min_samples_split,param_splitter,mean_fit_time,std_fit_time,mean_test_score,std_test_score
35,1,None,entropy,1,0.1,random,0.001701,0.000157,0.769818,0.030772
3,2,None,gini,1,0.1,random,0.001543,0.000058,0.766294,0.029910
127,3,balanced,entropy,0.3,0.3,random,0.001881,0.000381,0.759230,0.021145
61,3,None,entropy,0.3,0.2,random,0.001449,0.000143,0.759230,0.021145
120,3,balanced,entropy,0.3,2,best,0.002127,0.000079,0.759230,0.021145
...,...,...,...,...,...,...,...,...,...,...
97,124,balanced,entropy,1,2,random,0.003333,0.000518,0.704875,0.046323
32,125,None,entropy,1,2,best,0.007638,0.002861,0.703152,0.049379
69,126,balanced,gini,1,0.2,random,0.002155,0.000058,0.703090,0.031772
101,127,balanced,entropy,1,0.2,random,0.002078,0.000095,0.701335,0.037361


### Randomized Search

In [48]:
def randomizedSearchTree(X, y):
  crit_lst = ['gini', 'entropy']
  split_lst = ['best', 'random']
  minsplit_lst = [2, 0.1, 0.2, 0.3]
  minsamp_lst = [1, 0.1, 0.2, 0.3]
  classweight_lst = [None, 'balanced']

  grid_parametros = [
    {
        'criterion': crit_lst,
        'splitter': split_lst,
        'min_samples_split': minsplit_lst,
        'min_samples_leaf': minsamp_lst,
        'class_weight': classweight_lst
    }]

  tree = DecisionTreeClassifier()
  clf = RandomizedSearchCV(estimator=tree, param_distributions=grid_parametros, n_jobs=-1, n_iter=100)

  scaler = MinMaxScaler()
  clf = Pipeline([("scaler", scaler),
                  ("clf", clf)])
  
  clf.fit(X, y)

  return pd.DataFrame.from_dict(clf['clf'].cv_results_), clf, clf['clf'].best_score_

Sem binarização

In [49]:
aux = randomizedSearchTree(X_train0, y_train0)
listascores['Tree - sembin - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_splitter,param_min_samples_split,param_min_samples_leaf,param_criterion,param_class_weight,mean_fit_time,std_fit_time,mean_test_score,std_test_score
11,1,random,0.1,1,gini,None,0.001366,0.000062,0.780298,0.020195
12,2,best,0.1,1,entropy,None,0.002895,0.000349,0.775019,0.009541
84,3,random,0.2,0.1,entropy,None,0.001384,0.000158,0.771542,0.027064
5,4,random,0.1,0.1,entropy,None,0.001453,0.000180,0.769787,0.025517
26,4,random,0.3,1,entropy,None,0.001704,0.000409,0.769787,0.025517
...,...,...,...,...,...,...,...,...,...,...
2,96,best,0.2,1,entropy,balanced,0.003611,0.000550,0.725834,0.025650
96,97,random,2,1,entropy,balanced,0.002911,0.000163,0.724080,0.028570
72,98,random,0.3,1,entropy,balanced,0.001902,0.000052,0.717047,0.027391
49,99,best,0.2,1,gini,balanced,0.002384,0.000188,0.715200,0.038648


bin1_x

In [50]:
aux = randomizedSearchTree(X_train1, y_train1)
listascores['Tree - bin1_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_splitter,param_min_samples_split,param_min_samples_leaf,param_criterion,param_class_weight,mean_fit_time,std_fit_time,mean_test_score,std_test_score
56,1,random,0.1,1,gini,None,0.002838,0.002801,0.799674,0.011205
33,2,random,0.1,1,entropy,None,0.001454,0.000062,0.794411,0.022307
68,3,best,0.1,1,gini,None,0.002067,0.000184,0.783838,0.025719
57,4,random,0.2,0.1,entropy,None,0.002500,0.002430,0.775082,0.035561
83,5,random,0.3,0.1,entropy,None,0.001304,0.000040,0.773327,0.033676
...,...,...,...,...,...,...,...,...,...,...
35,93,best,0.3,0.1,gini,balanced,0.002997,0.001965,0.720602,0.032684
66,93,best,2,0.1,gini,balanced,0.002027,0.000019,0.720602,0.032684
53,93,best,2,0.1,entropy,balanced,0.002296,0.000117,0.720602,0.032684
49,99,best,0.3,1,entropy,balanced,0.002801,0.000210,0.720509,0.036138


bin2_x

In [51]:
aux = randomizedSearchTree(X_train2, y_train2)
listascores['Tree - bin2_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_splitter,param_min_samples_split,param_min_samples_leaf,param_criterion,param_class_weight,mean_fit_time,std_fit_time,mean_test_score,std_test_score
69,1,random,0.1,1,gini,None,0.001533,0.000063,0.783900,0.033194
43,2,random,0.1,1,entropy,None,0.002907,0.002798,0.768048,0.045108
22,3,random,0.1,1,entropy,balanced,0.004559,0.003025,0.762816,0.057932
0,4,best,0.1,1,entropy,None,0.004794,0.003387,0.762754,0.050174
15,5,best,2,1,gini,balanced,0.005514,0.002645,0.760984,0.016987
...,...,...,...,...,...,...,...,...,...,...
94,94,best,2,0.1,entropy,balanced,0.002346,0.000115,0.729390,0.032695
55,94,random,2,0.1,gini,balanced,0.001841,0.000035,0.729390,0.032695
19,94,best,0.3,0.1,gini,balanced,0.003178,0.002040,0.729390,0.032695
14,99,random,0.2,1,entropy,balanced,0.002227,0.000317,0.729343,0.027452


bin3_x

In [52]:
aux = randomizedSearchTree(X_train3, y_train3)
listascores['Tree - bin3_x - RandomizedSearch'] = (aux[1], aux[2])

get_results(aux[0])

,rank_test_score,param_splitter,param_min_samples_split,param_min_samples_leaf,param_criterion,param_class_weight,mean_fit_time,std_fit_time,mean_test_score,std_test_score
95,1,random,0.1,1,entropy,None,0.002750,0.001947,0.771588,0.043767
15,2,random,0.1,1,gini,None,0.001932,0.000647,0.771557,0.037869
0,3,random,0.3,0.2,entropy,None,0.003276,0.001631,0.759230,0.021145
46,3,best,0.2,0.2,gini,balanced,0.002072,0.000074,0.759230,0.021145
63,3,best,2,0.2,entropy,balanced,0.003481,0.001702,0.759230,0.021145
...,...,...,...,...,...,...,...,...,...,...
70,96,best,2,1,entropy,None,0.003847,0.000154,0.708384,0.039466
97,97,best,2,1,gini,None,0.002680,0.000126,0.708337,0.044775
85,98,random,0.2,1,entropy,balanced,0.002277,0.000220,0.706567,0.021840
91,99,random,2,1,entropy,balanced,0.004781,0.001824,0.702981,0.037007


# Escolha do algoritmo e geração do arquivo de saída

## Varredura de listascores, obtendo classificador com melhor score

In [53]:
max = ('', (0, 0))
for elem in listascores:
  if listascores[elem][1] > max[1][1]:
    max = (elem, listascores[elem])

print("max:", max[0])
print("score max:", max[1][1])
print("atributos:", max[1][0]['clf'].best_estimator_)

max: SVM - bin1_x - RandomizedSearch
score max: 0.8155100139729855
atributos: SVC(C=46.33865191768176, gamma=0.356456472427388)


Assim, o classificador de melhor score é o SVM, com a binarização bin1_x.

## Testando score

In [54]:
pipe = max[1][0] # Obtém o classificador melhor
pipe.score(X_test1, y_test1) # Verifica score contra conjunto de teste

0.7762237762237763

## Predição dos valores usando o classificador

In [55]:
Xpredict = bin1_Xpred.to_numpy()

print(Xpredict.shape)

(179, 10)


In [56]:
predicoes = pipe.predict(Xpredict)

listapred = ['sim'] * len(predicoes)

for i, val in enumerate(predicoes):
  if val == 0:
    listapred[i] = 'nao'
  else:
    listapred[i] = 'sim'

df_test['Sobreviveu'] = listapred

df_test.head()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"


,ClassePassageiro,Sexo,Idade,Parentes a Bordo,Pais / Crianças a Bordo,Ticket,Embarque,Sobreviveu
0,3,0,39.0,0,0,3101289,0,nao
1,1,1,51.0,1,0,13502,0,sim
2,2,1,40.0,1,1,29750,0,sim
3,3,0,36.0,0,0,0,0,nao
4,1,1,30.0,1,0,17569,1,sim


In [57]:
df_test["Sobreviveu"].to_csv("nave_submissaofinal.csv", index=False)